# Basic Samples : apache_age_py

## Prepare
```
import age
```
## Connect to PostgreSQL(with AGE extention)
* Connect to PostgreSQL server 
* Load AGE and register agtype to db session (Psycopg2 driver)
* Check graph exists and set graph. If not, age make that.

```
ag = age.connect(graph="(graph name}", host="{host}", port="{port}", dbname="{dbname}", user="{db username}", password="{password}")

# or 
DSN = "host={host} port={port} dbname={dbname} user={db username} password={password}"
ag = age.connect(graph="(graph name}", dsn=DSN)

# or Without Graph Name : you can make a new graph later.

ag = age.connect(host="{host}", port="{port}", dbname="{dbname}", user="{db username}", password="{password}")

# And set graph - if you don't have one yet, setGraph make that.)
ag = age.setGraph("{graph name}")
```

In [1]:
import age
from age.gen.ageParser import *

GRAPH_NAME = "test_graph"
DSN = "host=172.17.0.2 port=5432 dbname=postgres user=postgres password=agens"

ag = age.connect(graph=GRAPH_NAME, dsn=DSN)


---
# API

### age.connect(graph:str=None, dsn:str=None, connection_factory=None, cursor_factory=None, **kwargs) -> Age
> Connect PostgreSQL server 
  Parameters : dsn={dsn} or 
  host="{host}", port="{port}", dbname="{dbname}", user="{db username}", password="{password}"

### Age.commit(), Age.rollback(), Age.close()


### Age.execCypher(cypherStmt:str, commit:bool, *args) -> psycopg2.extensions.cursor :
> Execute statements to change data (CREATE, SET, REMOVE) without result.
>  If commit argument is True, commit automatically
>  If False, you must call 'Age.commit()' explicitly.
    
### Age.execCypherWithReturn(cypherStmt:str, columns:list, *args) -> psycopg2.extensions.cursor :
> Execute statements to change or retrieve data (CREATE, SET, REMOVE) with results.

>  If columns is None or empty array, it returns one agtype column result.

>  columns : str list \[ 'colName {type}', ... \] : If column data type is not set, agtype is default.

### Age.queryCypher(cypherStmt:str, columns:list, *args) -> psycopg2.extensions.cursor :
> Execute statements to retrieve data (CREATE, SET, REMOVE) with results.

>  If columns is None or empty array, it returns one agtype column result.

>  columns : str list \[ 'colName {type}', ... \] : If column data type is not set, agtype is default.
        

---
## Create  & Change Vertices

> If cypher statement changes data (create, set, remove), 
  you must use execCypher(cypherStmt, commit, *args). 
  
>  If **'commit'** argument is **True**: the cypherStmt make effect automatically, but cursor is closed after execution. So you cannot access the result.  
  If **False** : you can access the result, but you must commit session(ag.commit()) explicitly.
  (Otherwise the execution cannot make any effect.)


> execCypher(cypherStmt:str, commit:bool, *args) 

```
cursor = ag.execCypher("CREATE(...)", commit=False) # Cypher Create Statement
...
# check result in cursor
...
ag.commit() # commit explicitly
```


In [7]:
# Create Vertices
# Commit automatically
ag.execCypher("CREATE (n:Person {name: 'Joe'})", True)
cursor = ag.execCypher("CREATE (n:Person {name: 'Smith'})", True)
for row in cursor:
    print("NO RESULT: ", row[0])
    
# defer Commit 
cursor = ag.execCypher("CREATE (n:Person {name: %s}) RETURN n", False, ('Jack',))
for row in cursor:
    print("CREATED: ", row[0]) 
    
cursor = ag.execCypherWithReturn("CREATE (n:Person {name: %s, title: 'Developer'}) RETURN n", None, ('Andy',))
for row in cursor:
    print("CREATED: ", row[0])
    

cursor = ag.execCypherWithReturn("MATCH (n:Person {name: %s}) SET n.title=%s RETURN count(n)", ["a integer"], ('Smith','Manager',))
for row in cursor:
    print("SET: ", row[0])


cursor = ag.execCypherWithReturn("MATCH (n:Person {name: %s}) REMOVE n.title RETURN id(n)", ["a bigint"], ('Smith',))
for row in cursor:
    print("REMOVE Prop: ", row[0])

# You must commit explicitly
ag.commit()


CREATED:  {label:Person, id:844424930131997, properties:{name: Jack,}}::VERTEX
CREATED:  {label:Person, id:844424930131998, properties:{name: Andy,title: Developer,}}::VERTEX
SET:  8
REMOVE Prop:  844424930131970
REMOVE Prop:  844424930131973
REMOVE Prop:  844424930131976
REMOVE Prop:  844424930131980
REMOVE Prop:  844424930131984
REMOVE Prop:  844424930131988
REMOVE Prop:  844424930131992
REMOVE Prop:  844424930131996


---
## Query Vertices

> queryCypher(cypherStmt:str, columns:list, *args) 

### Single result column

```
cursor = ag.queryCypher("MATCH (n:Person {name: %s) RETURN n",[], 'Andy')
for row in cursor:
    vertex = row[0]
    print(vertex.id, vertex["name"], vertex) # row has id, label, properties 
```

### Multi result column

```
cursor = ag.queryCypher("MATCH (n:Person) RETURN label(n), n.name",['label VARCHAR', 'name'])
for row in cursor:
    label = row[0]
    name = row[1]
    print(label, name) 
```


### Vertex object has id, label attribute and __getitem__, __setitem__ for properties
```
vertex.id
vertex.label
vertex["property_name"]
```

In [ ]:

# Query Vertices with parsed row cursor.
print("-- Query Vertices  --------------------")
cursor = ag.queryCypher("MATCH (n:Person) RETURN n",[])
for row in cursor:
    vertex = row[0]
    print(vertex.id, vertex.label, vertex["name"])
    print("-->", vertex)

# Query Vertices with with multi column
print("-- Query Vertices with with multi columns. --------------------")
cursor = ag.queryCypher("MATCH (n:Person) RETURN label(n), n.name",['label VARCHAR', 'name'])
for row in cursor:
    label = row[0]
    name = row[1]
    print(label, name) 


---
## Create Relation 

> execCypher(cypherStmt:str, commit:bool, *args)


```
# Execute statement and handle results
cursor = ag.execCypher("MATCH (a:Person), (b:Person) WHERE  a.name = %s AND b.name = %s CREATE p=((a)-[r:workWith]->(b)) RETURN p", False, ('Andy', 'Smith',))
...
# You can access the results in cursor
...
ag.commit() # commit
```

```
# Auto commit
ag.execCypher("MATCH (a:Person), (b:Person) WHERE  a.name = 'Andy' AND b.name = 'Tom' CREATE (a)-[r:workWith]->(b)", True)

```


In [ ]:

# Create Edges
ag.execCypher("MATCH (a:Person), (b:Person) WHERE a.name = 'Joe' AND b.name = 'Smith' CREATE (a)-[r:workWith {weight: 3}]->(b)",True)
ag.execCypher("MATCH (a:Person), (b:Person) WHERE  a.name = 'Andy' AND b.name = 'Tom' CREATE (a)-[r:workWith {weight: 1}]->(b)",True)
ag.execCypher("MATCH (a:Person {name: 'Jack'}), (b:Person {name: 'Andy'}) CREATE (a)-[r:workWith {weight: 5}]->(b)",True)

# With Params
cursor = ag.execCypher("""MATCH (a:Person), (b:Person) 
        WHERE  a.name = %s AND b.name = %s 
        CREATE p=((a)-[r:workWith]->(b)) RETURN p""", False, 
        ('Andy', 'Smith',))

for row in cursor:
    print(row[0])
    
cursor = ag.execCypher("""MATCH (a:Person {name: 'Joe'}), (b:Person {name: 'Jack'}) 
        CREATE p=((a)-[r:workWith {weight: 5}]->(b))
        RETURN p """, True)


    

---
## Query Relations

> With single column
```
cursor = ag.queryCypher("MATCH p=()-[:workWith]-() RETURN p", None)
for row in cursor:
    path = row[0]
    print(path)  
```

> With multi columns
```
cursor = ag.queryCypher("MATCH p=(a)-[b]-(c) RETURN a,label(b),c", ["a","b VARCHAR","c"])
for row in cursor:
    start = row[0]
    edge = row[1]
    end = row[2]
    print(start["name"] , edge.label, end["name"])  
```


### Edge object has id, label,start_id, end_id attribute and __getitem__, __setitem__ for properties
```
edge = path.rel
edge.id
edge.label
edge.start_id
edge.end_id
edge["property_name"]
edge.properties
```

In [ ]:
cursor = ag.queryCypher("MATCH p=()-[:workWith]-() RETURN p", None)
for row in cursor:
    path = row[0]
    print("START:", path[0])
    print("EDGE:", path[1])
    print("END:", path[2])  

print("-- [Query path with multi columns --------")
cursor = ag.queryCypher("MATCH p=(a)-[b]-(c) WHERE b.weight>2 RETURN a,label(b), b.weight, c", ["a","bl","bw", "c"], (2,))
for row in cursor:
    start = row[0]
    edgel = row[1]
    edgew = row[2]
    end = row[3]
    print(start["name"] , edgel, edgew, end["name"]) 
    
    

## Query Scalar or properties value

In [ ]:
# Query scalar value
print("-- Query scalar value --------------------")
for row in ag.queryCypher("MATCH (n:Person) RETURN id(n)",None):
    print(row[0])
    
# Query properties 
print("-- Query properties --------------------")

for row in ag.queryCypher("MATCH (n:Person) RETURN properties(n)",None):
    print(row[0])
    
# Query properties value
print("-- Query property value --------------------")
for row in ag.queryCypher("MATCH (n:Person {name: 'Andy'}) RETURN n.title",None):
    print(row[0])
    
   

## Close connection

In [ ]:
# Clear test data
age.deleteGraph(ag.connection, GRAPH_NAME)
# connection close
ag.close()